# Sesi 5 – Orkestrator Multi-Agen

Menunjukkan pipeline dua agen sederhana (Peneliti -> Editor) menggunakan Foundry Local.


### Penjelasan: Instalasi Ketergantungan
Menginstal `foundry-local-sdk` dan `openai` yang diperlukan untuk akses model lokal dan penyelesaian obrolan. Idempoten.


# Skenario
Menerapkan pola orkestrator dua agen yang minimal:
- **Agen Peneliti** mengumpulkan poin-poin fakta yang ringkas
- **Agen Editor** menulis ulang untuk kejelasan eksekutif

Menunjukkan memori bersama per agen, pengoperan output sementara secara berurutan, dan fungsi pipeline sederhana. Dapat diperluas untuk lebih banyak peran (misalnya, Kritikus, Verifikator) atau cabang paralel.

**Variabel Lingkungan:**
- `FOUNDRY_LOCAL_ALIAS` - Model default yang digunakan (default: phi-4-mini)
- `AGENT_MODEL_PRIMARY` - Model agen utama (menggantikan ALIAS)
- `AGENT_MODEL_EDITOR` - Model agen editor (default ke model utama)

**Referensi SDK:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**Cara kerjanya:**
1. **FoundryLocalManager** secara otomatis memulai layanan Foundry Local
2. Mengunduh dan memuat model yang ditentukan (atau menggunakan versi yang telah di-cache)
3. Menyediakan endpoint yang kompatibel dengan OpenAI untuk interaksi
4. Setiap agen dapat menggunakan model yang berbeda untuk tugas khusus
5. Logika retry bawaan menangani kegagalan sementara dengan baik

**Fitur Utama:**
- ✅ Penemuan dan inisialisasi layanan otomatis
- ✅ Manajemen siklus hidup model (unduh, cache, muat)
- ✅ Kompatibilitas SDK OpenAI untuk API yang sudah dikenal
- ✅ Dukungan multi-model untuk spesialisasi agen
- ✅ Penanganan kesalahan yang kuat dengan logika retry
- ✅ Inferensi lokal (tidak memerlukan API cloud)


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### Penjelasan: Impor Inti & Pengetikan
Memperkenalkan dataclasses untuk penyimpanan pesan agen dan petunjuk pengetikan demi kejelasan. Mengimpor Foundry Local manager + OpenAI client untuk tindakan agen berikutnya.


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### Penjelasan: Inisialisasi Model (Pola SDK)
Menggunakan Foundry Local Python SDK untuk manajemen model yang andal:
- **FoundryLocalManager(alias)** - Secara otomatis memulai layanan dan memuat model berdasarkan alias
- **get_model_info(alias)** - Menyelesaikan alias menjadi ID model konkret
- **manager.endpoint** - Menyediakan endpoint layanan untuk klien OpenAI
- **manager.api_key** - Menyediakan API key (opsional untuk penggunaan lokal)
- Mendukung model terpisah untuk agen yang berbeda (utama vs editor)
- Logika retry bawaan dengan backoff eksponensial untuk ketahanan
- Verifikasi koneksi untuk memastikan layanan siap digunakan

**Pola SDK Utama:**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**Manajemen Siklus Hidup:**
- Manager disimpan secara global untuk pembersihan yang tepat
- Setiap agen dapat menggunakan model yang berbeda untuk spesialisasi
- Penemuan layanan otomatis dan penanganan koneksi
- Retry yang mulus dengan backoff eksponensial pada kegagalan

Hal ini memastikan inisialisasi yang tepat sebelum orkestrasi agen dimulai.

**Referensi:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### Penjelasan: Kelas Agent & Memory
Mendefinisikan `AgentMsg` yang ringan untuk entri memori dan `Agent` yang mencakup:
- **Peran sistem** - Persona dan instruksi agen
- **Riwayat pesan** - Menjaga konteks percakapan
- **Metode act()** - Menjalankan tindakan dengan penanganan kesalahan yang tepat

Agen dapat menggunakan model yang berbeda (utama vs editor) dan menjaga konteks yang terisolasi untuk setiap agen. Pola ini memungkinkan:
- Persistensi memori di antara tindakan
- Penugasan model yang fleksibel untuk setiap agen
- Isolasi dan pemulihan kesalahan
- Penggabungan dan orkestrasi yang mudah


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### Penjelasan: Pipeline yang Terorkestrasi
Membuat dua agen khusus:
- **Peneliti**: Menggunakan model utama, mengumpulkan informasi faktual
- **Editor**: Dapat menggunakan model terpisah (jika dikonfigurasi), menyempurnakan dan menulis ulang

Fungsi `pipeline`:
1. Peneliti mengumpulkan informasi mentah
2. Editor menyempurnakan menjadi output siap eksekutif
3. Mengembalikan hasil sementara dan hasil akhir

Pola ini memungkinkan:
- Spesialisasi model (model berbeda untuk peran yang berbeda)
- Peningkatan kualitas melalui pemrosesan multi-tahap
- Pelacakan transformasi informasi
- Ekstensi mudah ke lebih banyak agen atau pemrosesan paralel


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### Penjelasan: Eksekusi & Hasil Pipeline
Menjalankan pipeline multi-agen pada pertanyaan bertema kepatuhan + latensi untuk mendemonstrasikan:
- Transformasi informasi multi-tahap
- Spesialisasi dan kolaborasi antar agen
- Peningkatan kualitas output melalui penyempurnaan
- Keterlacakan (baik output sementara maupun akhir tetap terjaga)

**Struktur Hasil:**
- `question` - Pertanyaan asli dari pengguna
- `research` - Output riset mentah (poin-poin faktual)
- `final` - Ringkasan eksekutif yang telah disempurnakan
- `models` - Model yang digunakan pada setiap tahap

**Ide Pengembangan:**
1. Tambahkan agen Kritikus untuk meninjau kualitas
2. Terapkan agen riset paralel untuk aspek yang berbeda
3. Tambahkan agen Verifikator untuk pengecekan fakta
4. Gunakan model yang berbeda untuk tingkat kompleksitas yang berbeda
5. Terapkan loop umpan balik untuk perbaikan iteratif


### Lanjutan: Konfigurasi Agen Kustom

Cobalah menyesuaikan perilaku agen dengan mengubah variabel lingkungan sebelum menjalankan sel inisialisasi:

**Model yang Tersedia:**
- Gunakan `foundry model ls` di terminal untuk melihat semua model yang tersedia
- Contoh: phi-4-mini, phi-3.5-mini, qwen2.5-7b, llama-3.2-3b, dll.


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berupaya untuk memberikan hasil yang akurat, harap diperhatikan bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang berwenang. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa terjemahan manusia profesional. Kami tidak bertanggung jawab atas kesalahpahaman atau interpretasi yang salah yang timbul dari penggunaan terjemahan ini.
